In [119]:
import json
import ads
import orcid
import bibtexparser
import tqdm

ModuleNotFoundError: No module named 'tqdm'

## Generate Data from ORCID ##

In [25]:

orcid_access = json.load(open('keys.json'))
orcid_key = orcid_access['orcid_client_id']
orcid_secret = orcid_access['orcid_client_secret']
api = orcid.PublicAPI(orcid_key, orcid_secret)
token = api.get_search_token_from_orcid()

In [26]:
wkerzendorf_orcid = api.read_record_public('0000-0002-0479-7235', 'record', token)

In [32]:
json.dump(wkerzendorf_orcid, open('wkerzendorf_orcid.json', 'w'))

## Generating Publication statistics ##

In [123]:
ads_key = json.load(open('keys.json'))['ads_key']
ads.config.token = ads_key

In [160]:
ads.SearchQuery?

In [161]:
kerzendorf_query = ads.SearchQuery(q='author:"kerzendorf" database:astronomy', rows=1000)
papers = {}
for paper in kerzendorf_query:
    papers[paper.bibcode] = paper
print(kerzendorf_query.response.get_ratelimits())

{'limit': '5000', 'remaining': '4854', 'reset': '1541546847'}


In [163]:
whole_bibtex = ads.ExportQuery(list(papers.keys()), format='bibtex').execute()
with open('wekerzendorf_all.bib', 'w') as fh:
    fh.write(whole_bibtex)

In [164]:
bparser = bibtexparser.bparser.BibTexParser(interpolate_strings=False)

In [165]:
bibtex_db = bibtexparser.load(open('wekerzendorf_all.bib'), bparser)

In [166]:
for entry in bibtex_db.entries:
    cur_paper = papers[entry['ID']]
    refereed = True if 'REFEREED' in cur_paper.property else False
    entry['refereed'] = str(refereed)
    entry['citations'] = str(cur_paper.citation_count)

/Users/wkerzend/miniconda/envs/cv/lib/python3.7/site-packages/ads/utils.py:31: UserWarning: You are lazy loading attributes via 'property', and so are making multiple calls to the API. This will impact your overall rate limits.
  UserWarning,
/Users/wkerzend/miniconda/envs/cv/lib/python3.7/site-packages/ads/utils.py:31: UserWarning: You are lazy loading attributes via 'citation_count', and so are making multiple calls to the API. This will impact your overall rate limits.
  UserWarning,


In [167]:
bibtexparser.dump(bibtex_db, open('wekerzendorf_all_added.bib', 'w'))

In [168]:
!open -a bibdesk wekerzendorf_all_added.bib